## Conclusion from SQL and panda pre analysis
### Observation
* The code of a control area is unique. A control area will only have one unit and multiple SCPs but units may be duplicate among stations. Linename is messy. => Just use Control Area
* Time is not consistent too. => Use morning/afternoon/night...
* Get needed columns to save almost 100 mb
* Can keep desc-recovered but will need to clean duplicate data with both regular and recovered
* SQL query may be faster

### Completed
* Query data from database
* Clean duplicate data with both regular and recovered
* Create datetime frame

### To do
* Substract entries and exits to get the numbers
* Date and time analysis


In [2]:
#*****need to run*****
from sqlalchemy import create_engine
#from sqlalchemy import inspect
import pandas as pd

In [3]:
#*****need to run*****
engine = create_engine("sqlite:///mta.db")

In [4]:
#*****need to run*****
#df = pd.read_sql('SELECT ca, scp, station, date, time, desc, entries, exits FROM mta_data;', engine)
df = pd.read_sql('''SELECT ca, scp, station, date, time, desc, entries, exits FROM mta_data
                    WHERE date NOT IN ("02/27/2021", "02/28/2021", "06/01/2021", 
                                       "06/02/2021", "06/03/2021", "06/04/2021")
                    ORDER BY ca, date, time, scp, desc;''', engine)

In [5]:
#*****need to run*****
df["o_index"] = df.index
df["o_index"]
#drop duplicate data with wrong recover audit first
mask_dup = df[(df["o_index"] == 300706) |
          (df["o_index"] == 1089956) |
          (df["o_index"] == 1382146) |
          (df["o_index"] == 1382154) |
          ((df["CA"] == "N100") &
           (df["SCP"] == "00-00-00") &
           (df["ENTRIES"] == 818272) &
           (df["o_index"] != 842617))].index
df.drop(mask_dup, inplace= True)
#then, drop duplicate data with wrong regular data
df.drop_duplicates(subset=["CA", "SCP", "STATION", "DATE", "TIME"], keep= 'last', inplace=True)
df.drop(['o_index'], axis = 1, inplace = True)
df.reset_index(drop = True, inplace = True)
#datatime series
df["DATE_TIME"] = pd.to_datetime(df.DATE + " " + df.TIME, format="%m/%d/%Y %H:%M:%S")
df["DAY"] = df["DATE_TIME"].dt.weekday

In [5]:
df.sample(3)

,CA,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME,DAY
2210402,R251,00-00-02,96 ST,05/03/2021,21:00:00,REGULAR,17989622,4662556,2021-05-03 21:00:00,0
2079148,R228,00-00-04,28 ST,04/19/2021,09:00:00,REGULAR,405716,327612,2021-04-19 09:00:00,0
1916802,R168A,00-03-00,96 ST,03/08/2021,00:00:00,REGULAR,4723890,10637426,2021-03-08 00:00:00,0


In [6]:
dup = (df
       .groupby(["CA", "SCP", "STATION", "DATE", "TIME"])
       .ENTRIES.count()
       .reset_index()
       .sort_values("ENTRIES"))
dup[dup.ENTRIES > 1].shape

(0, 6)

In [7]:
df.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2747682 entries, 0 to 2747681
Data columns (total 10 columns):
 #   Column     Non-Null Count    Dtype         
---  ------     --------------    -----         
 0   CA         2747682 non-null  object        
 1   SCP        2747682 non-null  object        
 2   STATION    2747682 non-null  object        
 3   DATE       2747682 non-null  object        
 4   TIME       2747682 non-null  object        
 5   DESC       2747682 non-null  object        
 6   ENTRIES    2747682 non-null  int64         
 7   EXITS      2747682 non-null  int64         
 8   DATE_TIME  2747682 non-null  datetime64[ns]
 9   DAY        2747682 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 209.6+ MB


In [8]:
for col in df:
    print(col, df[col].nunique())

CA 750
SCP 224
STATION 379
DATE 92
TIME 56818
DESC 2
ENTRIES 1806445
EXITS 1717079
DATE_TIME 174383
DAY 7


In [9]:
for col in df:
    print(df[col].value_counts())

PTH22    23551
R610     13798
PTH03    13540
PTH02    12173
R238     12130
         ...  
R177      1440
J022      1104
H037      1104
A085       552
OB01       313
Name: CA, Length: 750, dtype: int64
00-00-00    254821
00-00-01    253109
00-00-02    225010
00-00-03    113927
00-03-00     99554
             ...  
00-04-09       521
02-02-04       520
03-01-07       508
03-00-08         3
01-FF-02         1
Name: SCP, Length: 224, dtype: int64
34 ST-PENN STA     56364
FULTON ST          53013
GRD CNTRL-42 ST    49022
23 ST              40300
86 ST              36002
                   ...  
HARLEM 148 ST       1640
CHAUNCEY ST         1640
CLEVELAND ST        1104
SUTTER AV           1104
ORCHARD BEACH        313
Name: STATION, Length: 379, dtype: int64
04/12/2021    30321
03/19/2021    30167
03/16/2021    30147
04/07/2021    30094
03/15/2021    30073
              ...  
04/29/2021    29800
05/23/2021    29789
05/02/2021    29783
05/14/2021    29761
03/14/2021    27156
Name: DATE, Lengt

In [9]:
#*********must run*********
#turnstiles_daily = (df.groupby(["CA", "SCP", "STATION", "DATE"], as_index=False).ENTRIES.last())
#(df.groupby(["CA", "SCP", "STATION", "DATE"], as_index=False).ENTRIES.max())
daily_max = (df
             .groupby(["CA", "SCP", "STATION", "DATE"], as_index=False)
             .agg({"ENTRIES":"max", "EXITS":"max"})
             .rename(columns={"ENTRIES": "ENTRIES_MAX", "EXITS": "EXITS_MAX"}))
daily_max

,CA,SCP,STATION,DATE,ENTRIES_MAX,EXITS_MAX
0,A002,02-00-00,59 ST,03/01/2021,7538400,2570968
1,A002,02-00-00,59 ST,03/02/2021,7538926,2571211
2,A002,02-00-00,59 ST,03/03/2021,7539503,2571472
3,A002,02-00-00,59 ST,03/04/2021,7540040,2571750
4,A002,02-00-00,59 ST,03/05/2021,7540626,2572022
...,...,...,...,...,...,...
459351,TRAM2,00-05-01,RIT-ROOSEVELT,05/27/2021,5554,577
459352,TRAM2,00-05-01,RIT-ROOSEVELT,05/28/2021,5554,577
459353,TRAM2,00-05-01,RIT-ROOSEVELT,05/29/2021,5554,577
459354,TRAM2,00-05-01,RIT-ROOSEVELT,05/30/2021,5554,577


In [11]:
#*********must run*********
#turnstiles_daily = (df.groupby(["CA", "SCP", "STATION", "DATE"], as_index=False).ENTRIES.first())
#turnstiles_daily = (df.groupby(["CA", "SCP", "STATION", "DATE"], as_index=False).ENTRIES.min())
daily_min = (df
             .groupby(["CA", "SCP", "STATION", "DATE"], as_index=False)
             .agg({"ENTRIES":"min", "EXITS":"min"})
             .rename(columns={"ENTRIES": "ENTRIES_MIN", "EXITS": "EXITS_MIN"}))
daily_min

,CA,SCP,STATION,DATE,ENTRIES_MIN,EXITS_MIN
0,A002,02-00-00,59 ST,03/01/2021,7537860,2570718
1,A002,02-00-00,59 ST,03/02/2021,7538409,2570969
2,A002,02-00-00,59 ST,03/03/2021,7538938,2571214
3,A002,02-00-00,59 ST,03/04/2021,7539514,2571474
4,A002,02-00-00,59 ST,03/05/2021,7540050,2571750
...,...,...,...,...,...,...
459351,TRAM2,00-05-01,RIT-ROOSEVELT,05/27/2021,5554,577
459352,TRAM2,00-05-01,RIT-ROOSEVELT,05/28/2021,5554,577
459353,TRAM2,00-05-01,RIT-ROOSEVELT,05/29/2021,5554,577
459354,TRAM2,00-05-01,RIT-ROOSEVELT,05/30/2021,5554,577


In [12]:
#*********must run*********
#SCP daily level
daily = pd.merge(daily_max, daily_min, on = ["CA", "SCP", "STATION", "DATE"])
daily["ENTRIES_DAILY"] = daily["ENTRIES_MAX"] - daily["ENTRIES_MIN"]
daily["EXITS_DAILY"] = daily["EXITS_MAX"] - daily["EXITS_MIN"]
daily["SUM_DAILY"] = daily["ENTRIES_DAILY"] + daily["EXITS_DAILY"]
daily["DATE"] = pd.to_datetime(daily.DATE, format="%m/%d/%Y")
daily["DAY"] = daily["DATE"].dt.weekday
daily

,CA,SCP,STATION,DATE,ENTRIES_MAX,EXITS_MAX,ENTRIES_MIN,EXITS_MIN,ENTRIES_DAILY,EXITS_DAILY,SUM_DAILY,DAY
0,A002,02-00-00,59 ST,2021-03-01,7538400,2570968,7537860,2570718,540,250,790,0
1,A002,02-00-00,59 ST,2021-03-02,7538926,2571211,7538409,2570969,517,242,759,1
2,A002,02-00-00,59 ST,2021-03-03,7539503,2571472,7538938,2571214,565,258,823,2
3,A002,02-00-00,59 ST,2021-03-04,7540040,2571750,7539514,2571474,526,276,802,3
4,A002,02-00-00,59 ST,2021-03-05,7540626,2572022,7540050,2571750,576,272,848,4
...,...,...,...,...,...,...,...,...,...,...,...,...
459351,TRAM2,00-05-01,RIT-ROOSEVELT,2021-05-27,5554,577,5554,577,0,0,0,3
459352,TRAM2,00-05-01,RIT-ROOSEVELT,2021-05-28,5554,577,5554,577,0,0,0,4
459353,TRAM2,00-05-01,RIT-ROOSEVELT,2021-05-29,5554,577,5554,577,0,0,0,5
459354,TRAM2,00-05-01,RIT-ROOSEVELT,2021-05-30,5554,577,5554,577,0,0,0,6


In [61]:
daily[daily["ENTRIES_DAILY"] < 0]

,CA,SCP,STATION,DATE,ENTRIES_MAX,EXITS_MAX,ENTRIES_MIN,EXITS_MIN,ENTRIES_DAILY,EXITS_DAILY,SUM_DAILY,DAY
389,A002,02-03-02,59 ST,2021-03-22,238,88,6641438,8869481,-6641200,-8869393,-15510593,0
3588,A011,01-03-00,57 ST-7 AV,2021-03-01,885620021,489657460,885620231,489658301,-210,-841,-1051,0
3589,A011,01-03-00,57 ST-7 AV,2021-03-02,885619834,489656586,885619997,489657439,-163,-853,-1016,1
3590,A011,01-03-00,57 ST-7 AV,2021-03-03,885619673,489655757,885619808,489656568,-135,-811,-946,2
3591,A011,01-03-00,57 ST-7 AV,2021-03-04,885619386,489654766,885619620,489655681,-234,-915,-1149,3
...,...,...,...,...,...,...,...,...,...,...,...,...
455489,R730,00-00-04,EASTCHSTER/DYRE,2021-05-29,1559810360,1728247375,1559810397,1728247349,-37,26,-11,5
455490,R730,00-00-04,EASTCHSTER/DYRE,2021-05-30,1559810330,1728247391,1559810354,1728247375,-24,16,-8,6
455491,R730,00-00-04,EASTCHSTER/DYRE,2021-05-31,1559810287,1728247429,1559810329,1728247394,-42,35,-7,0
455891,S101,00-00-04,ST. GEORGE,2021-04-01,25,0,980934,174,-980909,-174,-981083,3


In [62]:
daily[daily["EXITS_DAILY"] < 0]

,CA,SCP,STATION,DATE,ENTRIES_MAX,EXITS_MAX,ENTRIES_MIN,EXITS_MIN,ENTRIES_DAILY,EXITS_DAILY,SUM_DAILY,DAY
389,A002,02-03-02,59 ST,2021-03-22,238,88,6641438,8869481,-6641200,-8869393,-15510593,0
3588,A011,01-03-00,57 ST-7 AV,2021-03-01,885620021,489657460,885620231,489658301,-210,-841,-1051,0
3589,A011,01-03-00,57 ST-7 AV,2021-03-02,885619834,489656586,885619997,489657439,-163,-853,-1016,1
3590,A011,01-03-00,57 ST-7 AV,2021-03-03,885619673,489655757,885619808,489656568,-135,-811,-946,2
3591,A011,01-03-00,57 ST-7 AV,2021-03-04,885619386,489654766,885619620,489655681,-234,-915,-1149,3
...,...,...,...,...,...,...,...,...,...,...,...,...
442611,R622,00-00-00,FRANKLIN AV,2021-05-30,262380696,2062018712,262381338,2062018949,-642,-237,-879,6
442612,R622,00-00-00,FRANKLIN AV,2021-05-31,262379874,2062018360,262380625,2062018655,-751,-295,-1046,0
443727,R624,00-00-02,KINGSTON AV,2021-03-11,577,469,11839228,8157517,-11838651,-8157048,-19995699,3
455891,S101,00-00-04,ST. GEORGE,2021-04-01,25,0,980934,174,-980909,-174,-981083,3


In [67]:
mask = ((df["CA"] == "J003") & 
(df["SCP"] == "00-00-00") &
(df["DATE"] == "04/16/2021"))

df[mask]

,CA,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME,DAY
517000,J003,00-00-00,HEWES ST,04/16/2021,01:00:00,REGULAR,8421831,2645622,2021-04-16 01:00:00,4
517007,J003,00-00-00,HEWES ST,04/16/2021,05:00:00,REGULAR,8421831,2645624,2021-04-16 05:00:00,4
517014,J003,00-00-00,HEWES ST,04/16/2021,09:00:00,REGULAR,8421832,2645683,2021-04-16 09:00:00,4
517021,J003,00-00-00,HEWES ST,04/16/2021,13:00:00,REGULAR,8421832,2645729,2021-04-16 13:00:00,4
517028,J003,00-00-00,HEWES ST,04/16/2021,17:00:00,REGULAR,2147432317,671146375,2021-04-16 17:00:00,4
517035,J003,00-00-00,HEWES ST,04/16/2021,21:00:00,REGULAR,2147432315,671146416,2021-04-16 21:00:00,4


In [79]:
mask_outlier = daily[(daily["ENTRIES_DAILY"] < 0) |
                     (daily["EXITS_DAILY"] < 0) |
                     (daily["ENTRIES_DAILY"] > 1000000) |
                     (daily["EXITS_DAILY"] > 1000000)].index
daily.drop(mask_outlier, inplace= True)
daily.sort_values("SUM_DAILY", ascending=False)

,CA,SCP,STATION,DATE,ENTRIES_MAX,EXITS_MAX,ENTRIES_MIN,EXITS_MIN,ENTRIES_DAILY,EXITS_DAILY,SUM_DAILY,DAY
58252,D006,01-05-00,NEW UTRECHT AV,2021-04-11,589827,0,953,0,588874,0,588874,6
338140,R205A,04-05-00,FULTON ST,2021-03-11,327681,65536,0,0,327681,65536,393217,3
288221,R101,02-06-05,SOUTH FERRY,2021-03-17,327681,65537,1,83,327680,65454,393134,2
72947,H007A,02-05-00,1 AV,2021-03-22,524292,63,262144,0,262148,63,262211,0
73040,H007A,02-05-01,1 AV,2021-03-23,524311,66,262147,27,262164,39,262203,1
...,...,...,...,...,...,...,...,...,...,...,...,...
403036,R418,00-06-01,CASTLE HILL AV,2021-04-13,3,248,3,248,0,0,0,1
403037,R418,00-06-01,CASTLE HILL AV,2021-04-14,3,248,3,248,0,0,0,2
403038,R418,00-06-01,CASTLE HILL AV,2021-04-15,3,248,3,248,0,0,0,3
403039,R418,00-06-01,CASTLE HILL AV,2021-04-16,3,248,3,248,0,0,0,4


In [80]:
mask = ((df["CA"] == "D006") & 
(df["SCP"] == "01-05-00") &
(df["DATE"] == "04/11/2021"))

df[mask]

,CA,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME,DAY
350189,D006,01-05-00,NEW UTRECHT AV,04/11/2021,01:00:00,REGULAR,953,0,2021-04-11 01:00:00,6
350193,D006,01-05-00,NEW UTRECHT AV,04/11/2021,05:00:00,REGULAR,953,0,2021-04-11 05:00:00,6
350197,D006,01-05-00,NEW UTRECHT AV,04/11/2021,09:00:00,REGULAR,589827,0,2021-04-11 09:00:00,6
350201,D006,01-05-00,NEW UTRECHT AV,04/11/2021,13:00:00,REGULAR,589827,0,2021-04-11 13:00:00,6
350205,D006,01-05-00,NEW UTRECHT AV,04/11/2021,17:00:00,REGULAR,589827,0,2021-04-11 17:00:00,6
350209,D006,01-05-00,NEW UTRECHT AV,04/11/2021,21:00:00,REGULAR,589827,0,2021-04-11 21:00:00,6


In [58]:
mask = ((df["CA"] == "D006") & 
(df["SCP"] == "01-05-00") &
(df["DATE"] == "04/11/2021"))

df[mask]

,CA,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME,DAY
0,A002,02-00-00,59 ST,03/01/2021,03:00:00,REGULAR,7537860,2570718,2021-03-01 03:00:00,0
12,A002,02-00-00,59 ST,03/01/2021,07:00:00,REGULAR,7537862,2570742,2021-03-01 07:00:00,0
24,A002,02-00-00,59 ST,03/01/2021,11:00:00,REGULAR,7537914,2570858,2021-03-01 11:00:00,0
36,A002,02-00-00,59 ST,03/01/2021,15:00:00,REGULAR,7538016,2570907,2021-03-01 15:00:00,0
48,A002,02-00-00,59 ST,03/01/2021,19:00:00,REGULAR,7538294,2570952,2021-03-01 19:00:00,0
60,A002,02-00-00,59 ST,03/01/2021,23:00:00,REGULAR,7538400,2570968,2021-03-01 23:00:00,0


In [81]:
mask = ((df["CA"] == "H007A") & 
(df["SCP"] == "02-05-00") &
(df["DATE"] == "03/22/2021"))

df[mask]

,CA,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME,DAY
432881,H007A,02-05-00,1 AV,03/22/2021,00:00:00,REGULAR,262144,0,2021-03-22 00:00:00,0
432895,H007A,02-05-00,1 AV,03/22/2021,04:00:00,REGULAR,262144,0,2021-03-22 04:00:00,0
432909,H007A,02-05-00,1 AV,03/22/2021,08:00:00,REGULAR,262144,0,2021-03-22 08:00:00,0
432923,H007A,02-05-00,1 AV,03/22/2021,12:00:00,REGULAR,262144,0,2021-03-22 12:00:00,0
432937,H007A,02-05-00,1 AV,03/22/2021,16:00:00,REGULAR,262147,3,2021-03-22 16:00:00,0
432951,H007A,02-05-00,1 AV,03/22/2021,20:00:00,REGULAR,524292,63,2021-03-22 20:00:00,0


In [83]:
mask_outlier = daily[(daily["ENTRIES_DAILY"] < 0) |
                     (daily["EXITS_DAILY"] < 0) |
                     (daily["ENTRIES_DAILY"] > 200000) |
                     (daily["EXITS_DAILY"] > 200000)].index
daily.drop(mask_outlier, inplace= True)
daily.sort_values("SUM_DAILY", ascending=False)

,CA,SCP,STATION,DATE,ENTRIES_MAX,EXITS_MAX,ENTRIES_MIN,EXITS_MIN,ENTRIES_DAILY,EXITS_DAILY,SUM_DAILY,DAY
266711,PTH07,00-00-00,CITY / BUS,2021-05-31,164655,132500,47229,105923,117426,26577,144003,0
318725,R161B,00-05-02,72 ST,2021-05-08,393216,0,327680,0,65536,0,65536,5
371918,R258,00-00-00,125 ST,2021-05-12,44861,21541,12472,12496,32389,9045,41434,2
272921,PTH13,00-04-08,THIRTY ST,2021-04-21,36,60017,10,22758,26,37259,37285,2
280862,PTH20,03-00-08,NEWARK HM HE,2021-04-14,1084,35740,24,6,1060,35734,36794,2
...,...,...,...,...,...,...,...,...,...,...,...,...
399535,R413,00-05-00,WHITLOCK AV,2021-04-04,458792,0,458792,0,0,0,0,6
399536,R413,00-05-00,WHITLOCK AV,2021-04-05,458792,0,458792,0,0,0,0,0
399537,R413,00-05-00,WHITLOCK AV,2021-04-06,458792,0,458792,0,0,0,0,1
399538,R413,00-05-00,WHITLOCK AV,2021-04-07,458792,0,458792,0,0,0,0,2


In [84]:
mask = ((df["CA"] == "PTH07") & 
(df["SCP"] == "00-05-02") &
(df["DATE"] == "05/31/2021"))

df[mask]

,CA,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME,DAY
1608469,PTH07,00-00-00,CITY / BUS,05/31/2021,01:34:41,REGULAR,47229,105923,2021-05-31 01:34:41,0
1608474,PTH07,00-00-00,CITY / BUS,05/31/2021,03:07:20,REGULAR,164610,132374,2021-05-31 03:07:20,0
1608497,PTH07,00-00-00,CITY / BUS,05/31/2021,07:19:20,REGULAR,164611,132375,2021-05-31 07:19:20,0
1608520,PTH07,00-00-00,CITY / BUS,05/31/2021,11:31:20,REGULAR,164612,132383,2021-05-31 11:31:20,0
1608542,PTH07,00-00-00,CITY / BUS,05/31/2021,15:43:20,REGULAR,164640,132426,2021-05-31 15:43:20,0
1608565,PTH07,00-00-00,CITY / BUS,05/31/2021,19:55:20,REGULAR,164655,132500,2021-05-31 19:55:20,0


In [86]:
mask = ((df["CA"] == "R161B") & 
(df["SCP"] == "00-05-02") &
(df["DATE"] == "05/08/2021"))

df[mask]

,CA,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME,DAY
1902085,R161B,00-05-02,72 ST,05/08/2021,01:00:00,REGULAR,327680,0,2021-05-08 01:00:00,5
1902097,R161B,00-05-02,72 ST,05/08/2021,05:00:00,REGULAR,327680,0,2021-05-08 05:00:00,5
1902109,R161B,00-05-02,72 ST,05/08/2021,09:00:00,REGULAR,327680,0,2021-05-08 09:00:00,5
1902121,R161B,00-05-02,72 ST,05/08/2021,13:00:00,REGULAR,393216,0,2021-05-08 13:00:00,5
1902133,R161B,00-05-02,72 ST,05/08/2021,17:00:00,RECOVR AUD,393216,0,2021-05-08 17:00:00,5
1902145,R161B,00-05-02,72 ST,05/08/2021,21:00:00,RECOVR AUD,393216,0,2021-05-08 21:00:00,5


In [88]:
mask_outlier = daily[(daily["ENTRIES_DAILY"] < 0) |
                     (daily["EXITS_DAILY"] < 0) |
                     (daily["ENTRIES_DAILY"] > 36000) |
                     (daily["EXITS_DAILY"] > 36000)].index
daily.drop(mask_outlier, inplace= True)
daily.sort_values("SUM_DAILY", ascending=False)

,CA,SCP,STATION,DATE,ENTRIES_MAX,EXITS_MAX,ENTRIES_MIN,EXITS_MIN,ENTRIES_DAILY,EXITS_DAILY,SUM_DAILY,DAY
371918,R258,00-00-00,125 ST,2021-05-12,44861,21541,12472,12496,32389,9045,41434,2
422126,R528,00-00-04,JUNCTION BLVD,2021-03-13,3815836,1498295,3814012,1493436,1824,4859,6683,5
422140,R528,00-00-04,JUNCTION BLVD,2021-03-27,3843513,1528232,3841994,1524025,1519,4207,5726,5
422133,R528,00-00-04,JUNCTION BLVD,2021-03-20,3829071,1511337,3827558,1507353,1513,3984,5497,5
371926,R258,00-00-00,125 ST,2021-05-20,49509,44166,47842,40340,1667,3826,5493,3
...,...,...,...,...,...,...,...,...,...,...,...,...
94517,JFK02,01-00-05,HOWARD BCH JFK,2021-05-15,117900,331143,117900,331143,0,0,0,5
94518,JFK02,01-00-05,HOWARD BCH JFK,2021-05-16,117900,331143,117900,331143,0,0,0,6
326852,R179,01-00-00,157 ST,2021-03-21,4227287,7232800,4227287,7232800,0,0,0,6
326845,R179,01-00-00,157 ST,2021-03-14,4224399,7227389,4224399,7227389,0,0,0,6


In [89]:
mask = ((df["CA"] == "R258") & 
(df["SCP"] == "00-00-00") &
(df["DATE"] == "05/12/2021"))

df[mask]

,CA,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME,DAY
2228342,R258,00-00-00,125 ST,05/12/2021,01:00:00,REGULAR,12472,12496,2021-05-12 01:00:00,2
2228355,R258,00-00-00,125 ST,05/12/2021,05:00:00,REGULAR,12472,12521,2021-05-12 05:00:00,2
2228368,R258,00-00-00,125 ST,05/12/2021,09:00:00,REGULAR,12620,12973,2021-05-12 09:00:00,2
2228381,R258,00-00-00,125 ST,05/12/2021,13:00:00,REGULAR,12719,13431,2021-05-12 13:00:00,2
2228394,R258,00-00-00,125 ST,05/12/2021,17:00:00,REGULAR,44859,21100,2021-05-12 17:00:00,2
2228407,R258,00-00-00,125 ST,05/12/2021,21:00:00,REGULAR,44861,21541,2021-05-12 21:00:00,2


In [13]:
#*********must run*********
mask_outlier = daily[(daily["ENTRIES_DAILY"] < 0) |
                     (daily["EXITS_DAILY"] < 0) |
                     (daily["ENTRIES_DAILY"] > 30000) |
                     (daily["EXITS_DAILY"] > 30000)].index
daily.drop(mask_outlier, inplace= True)
daily.sort_values("SUM_DAILY", ascending=False)

,CA,SCP,STATION,DATE,ENTRIES_MAX,EXITS_MAX,ENTRIES_MIN,EXITS_MIN,ENTRIES_DAILY,EXITS_DAILY,SUM_DAILY,DAY
283334,PTH22,00-01-04,PATH NEW WTC,2021-03-05,26234,24186,45,222,26189,23964,50153,4
261642,PTH03,00-01-08,JOURNAL SQUARE,2021-04-23,27171,17555,1,0,27170,17555,44725,4
261641,PTH03,00-01-08,JOURNAL SQUARE,2021-04-22,26948,17489,1,0,26947,17489,44436,3
315867,R160,02-03-00,66 ST-LINCOLN,2021-04-25,28817,15367,13,3,28804,15364,44168,6
261635,PTH03,00-01-08,JOURNAL SQUARE,2021-04-16,26237,17112,1,0,26236,17112,43348,4
...,...,...,...,...,...,...,...,...,...,...,...,...
316159,R160,02-05-01,66 ST-LINCOLN,2021-05-11,0,927,0,927,0,0,0,1
316158,R160,02-05-01,66 ST-LINCOLN,2021-05-10,0,927,0,927,0,0,0,0
316157,R160,02-05-01,66 ST-LINCOLN,2021-05-09,0,927,0,927,0,0,0,6
316156,R160,02-05-01,66 ST-LINCOLN,2021-05-08,0,927,0,927,0,0,0,5


In [93]:
#CA daily level
ca_daily = (daily
                 .groupby(["CA", "STATION", "DATE", "DAY"], as_index=False)
                 .agg({"SUM_DAILY":"sum", "ENTRIES_DAILY":"sum", "EXITS_DAILY":"sum"}))
ca_daily

,CA,STATION,DATE,DAY,SUM_DAILY,ENTRIES_DAILY,EXITS_DAILY
0,A002,59 ST,2021-03-01,0,7494,4099,3395
1,A002,59 ST,2021-03-02,1,7845,4336,3509
2,A002,59 ST,2021-03-03,2,8219,4447,3772
3,A002,59 ST,2021-03-04,3,7982,4376,3606
4,A002,59 ST,2021-03-05,4,8045,4436,3609
...,...,...,...,...,...,...,...
68911,TRAM2,RIT-ROOSEVELT,2021-05-27,3,1975,1940,35
68912,TRAM2,RIT-ROOSEVELT,2021-05-28,4,1638,1608,30
68913,TRAM2,RIT-ROOSEVELT,2021-05-29,5,1347,1311,36
68914,TRAM2,RIT-ROOSEVELT,2021-05-30,6,1068,1029,39


In [94]:
#station daily level
station_daily = (daily
                 .groupby(["STATION", "DATE", "DAY"], as_index=False)
                 .agg({"SUM_DAILY":"sum", "ENTRIES_DAILY":"sum", "EXITS_DAILY":"sum"}))
station_daily

,STATION,DATE,DAY,SUM_DAILY,ENTRIES_DAILY,EXITS_DAILY
0,1 AV,2021-03-01,0,15040,5924,9116
1,1 AV,2021-03-02,1,15688,6085,9603
2,1 AV,2021-03-03,2,16251,6365,9886
3,1 AV,2021-03-04,3,16202,6290,9912
4,1 AV,2021-03-05,4,17031,6377,10654
...,...,...,...,...,...,...
34779,ZEREGA AV,2021-05-27,3,2347,1130,1217
34780,ZEREGA AV,2021-05-28,4,2218,1056,1162
34781,ZEREGA AV,2021-05-29,5,1138,531,607
34782,ZEREGA AV,2021-05-30,6,875,400,475


In [95]:
#station weekday level
station_weekday = (daily
                 .groupby(["STATION", "DAY"], as_index=False)
                 .agg({"SUM_DAILY":"sum", "ENTRIES_DAILY":"sum", "EXITS_DAILY":"sum"}))
station_weekday

,STATION,DAY,SUM_DAILY,ENTRIES_DAILY,EXITS_DAILY
0,1 AV,0,210223,79169,131054
1,1 AV,1,209323,78851,130472
2,1 AV,2,212659,79663,132996
3,1 AV,3,213262,80836,132426
4,1 AV,4,224496,83151,141345
...,...,...,...,...,...
2648,ZEREGA AV,2,28558,13604,14954
2649,ZEREGA AV,3,28029,13489,14540
2650,ZEREGA AV,4,27726,13203,14523
2651,ZEREGA AV,5,13276,6237,7039


In [14]:
#station 3_month level
station_3_month = (daily
                 .groupby(["STATION"], as_index=False)
                 .agg({"SUM_DAILY":"sum", "ENTRIES_DAILY":"sum", "EXITS_DAILY":"sum"}))
station_3_month.sort_values("SUM_DAILY", ascending=False)

,STATION,SUM_DAILY,ENTRIES_DAILY,EXITS_DAILY
61,34 ST-PENN STA,6467822,2747128,3720694
59,34 ST-HERALD SQ,5161520,2194722,2966798
110,86 ST,4963336,2182962,2780374
233,GRD CNTRL-42 ST,4673949,1956992,2716957
9,125 ST,4583501,2067826,2515675
...,...,...,...,...
148,BEACH 98 ST,24142,10163,13979
354,TOMPKINSVILLE,22476,22476,0
141,BEACH 105 ST,13888,5046,8842
162,BROAD CHANNEL,12269,8386,3883


In [16]:
station_3_month.sort_values("SUM_DAILY", ascending=False).head(20)

,STATION,SUM_DAILY,ENTRIES_DAILY,EXITS_DAILY
61,34 ST-PENN STA,6467822,2747128,3720694
59,34 ST-HERALD SQ,5161520,2194722,2966798
110,86 ST,4963336,2182962,2780374
233,GRD CNTRL-42 ST,4673949,1956992,2716957
9,125 ST,4583501,2067826,2515675
14,14 ST-UNION SQ,4518479,1856868,2661611
68,42 ST-PORT AUTH,4223261,1937073,2286188
46,23 ST,4089563,1889427,2200136
85,59 ST,3529918,1595818,1934100
226,FULTON ST,3514154,1535041,1979113
